In [6]:
#normal check whats available and all
import concurrent.futures
import pandas as pd
import tensorflow as tf
import time

# Initialize TPU if needed
def init_tpu():
    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print("TPU initialized.")
        return True
    except Exception as e:
        print(f"TPU initialization failed: {e}")
        return False

def load_keywords_from_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        print("Columns in the Excel file:", df.columns.tolist())  # Print the columns to check
        df.columns = df.columns.str.strip()  # Strip whitespace from column names

        # Create a list to hold keywords and their corresponding resource allocations
        keywords = []
        for _, row in df.iterrows():
            resource = (int(row['CPU']), int(row['GPU']), int(row['TPU']))
            keywords.append((row['Keyword'], *resource))  # Add keyword and resource tuple to the list

        print(f"Loaded keywords: {keywords}")  # Debug print to check the structure of keywords
        return keywords  # Return the list of keywords
    except Exception as e:
        print(f"An error occurred: {e}")
        return None  # Return None in case of an error

# Load CNN snippets to be executed
cnn_snippets = [
    """import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model""",

    """def compile_and_train_model(model, train_images, train_labels, epochs=10):
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(train_images, train_labels, epochs=epochs)""",

    """def evaluate_model(model, test_images, test_labels):
        test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
        print(f'\\nTest accuracy: {test_acc}')""",

    """def load_and_preprocess_data():
        (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
        train_images = train_images.astype('float32') / 255.0
        test_images = test_images.astype('float32') / 255.0
        return train_images, train_labels, test_images, test_labels"""
]

# Default resource allocation
default_resource = (1, 0, 0)  # CPU, GPU, TPU

def label_and_allocate(snippet, keywords):
    """Label the code snippet and allocate resources based on keywords."""
    resources = []
    for keyword, cpu, gpu, tpu in keywords:
        if keyword in snippet:
            resource = {
                'keyword': keyword,
                'CPU': bool(cpu),
                'GPU': bool(gpu),
                'TPU': bool(tpu)
            }
            resources.append(resource)
    return resources

# Function to run each CNN snippet
def run_snippet(snippet, device):
    print(f"Executing snippet: {snippet[:30]}... on {device}")  # Print a preview of the snippet and the device
    start_time = time.time()  # Start timing the execution
    with tf.device(device):  # Allocate device for this snippet
        exec(snippet)  # Execute the snippet
    execution_time = time.time() - start_time  # Calculate execution time
    return f"Execution completed in {execution_time:.2f} seconds on {device}."




def get_device(cpu, gpu, tpu):
    """Determine the device to use based on resource availability."""
    if tpu and tf.config.list_logical_devices('TPU'):
        return '/device:TPU:0'
    elif gpu and tf.config.list_physical_devices('GPU'):
        return '/device:GPU:0'
    else:
        return '/device:CPU:0'

def master(file_path):
    keywords = load_keywords_from_excel(file_path)
    if keywords is None:
        print("Failed to load keywords.")
        return  # Stop further execution if keywords are not loaded

    print("Loaded keywords:", keywords)  # Debug print to show loaded keywords
    labeled_resources = []

    # Label each snippet and allocate resources
    for snippet in cnn_snippets:
        resources = label_and_allocate(snippet, keywords)
        labeled_resources.append((resources, snippet))

    # Initialize TPU if any task requires it
    tpu_required = any(resource[-1] for resource in keywords)  # Check if TPU is required (third item in the tuple)
    if tpu_required:
        tpu_available = init_tpu()  # Initialize TPU if needed
        if not tpu_available:
            print("TPU is required but unavailable. Proceeding with fallback to GPU or CPU.")

    # Using ThreadPoolExecutor to simulate slave execution
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        futures = {}
        for resources, snippet in labeled_resources:
            if resources:
                # Only extract CPU, GPU, and TPU values (ignore the keyword)
                resource_values = list(resources[0].values())[1:]  # Extract CPU, GPU, TPU values only
                device = get_device(*resource_values)
            else:
                device = '/device:CPU:0'  # Default to CPU if no resource is matched

            futures[executor.submit(run_snippet, snippet, device)] = resources

        for future in concurrent.futures.as_completed(futures):
            resources = futures[future]
            try:
                result = future.result()
                print(f"Resources used: {resources} -> {result}")
            except Exception as e:
                print(f"Error executing snippet: {e}")




if __name__ == '__main__':
    # Specify the path to your Excel file
    excel_file_path = '/content/resource_allocation_dataset.xlsx'
    master(excel_file_path)


Columns in the Excel file: ['Keyword', 'CPU', 'GPU', 'TPU']
Loaded keywords: [('default', 1, 0, 0), ('import', 1, 0, 0), ('CNN', 0, 1, 1), ('RNN', 1, 0, 1), ('preprocess', 1, 1, 0), ('train', 0, 1, 1), ('eval', 1, 0, 0), ('transfer', 0, 1, 1), ('feature', 1, 1, 0), ('tune', 1, 0, 0), ('grid', 1, 0, 0), ('LSTM', 1, 0, 1), ('data_augmentation', 1, 1, 0), ('batch_normalization', 0, 1, 1), ('dropout', 1, 0, 0), ('optimization', 1, 1, 1), ('model_saving', 1, 0, 0), ('hyperparameter_search', 1, 1, 0), ('data_loading', 1, 1, 0), ('evaluation_metrics', 1, 0, 0), ('fine_tuning', 0, 1, 1), ('ensemble_learning', 1, 1, 0), ('tokenization', 1, 0, 0), ('backpropagation', 1, 0, 0), ('regularization', 1, 0, 0), ('transfer_learning', 0, 1, 1), ('seq2seq', 1, 0, 1), ('attention_mechanism', 0, 1, 1), ('model_export', 1, 0, 0), ('real_time_inference', 0, 1, 1), ('predictive_analysis', 1, 0, 0), ('class_weighting', 1, 1, 0), ('grid_search_cv', 1, 0, 0), ('cross_validation', 1, 1, 0), ('feature_scaling', 1,

In [7]:
#error in executing the snippet
import concurrent.futures
import pandas as pd
import tensorflow as tf
import time

# Initialize TPU if needed
def init_tpu():
    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print("TPU initialized.")
        return True
    except Exception as e:
        print(f"TPU initialization failed: {e}")
        return False

def load_keywords_from_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        print("Columns in the Excel file:", df.columns.tolist())  # Print the columns to check
        df.columns = df.columns.str.strip()  # Strip whitespace from column names

        # Create a list to hold keywords and their corresponding resource allocations
        keywords = []
        for _, row in df.iterrows():
            resource = (int(row['CPU']), int(row['GPU']), int(row['TPU']))
            keywords.append((row['Keyword'], *resource))  # Add keyword and resource tuple to the list

        print(f"Loaded keywords: {keywords}")  # Debug print to check the structure of keywords
        return keywords  # Return the list of keywords
    except Exception as e:
        print(f"An error occurred: {e}")
        return None  # Return None in case of an error

# Load CNN snippets to be executed
cnn_snippets = [
    """import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model""",

    """def compile_and_train_model(model, train_images, train_labels, epochs=10):
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(train_images, train_labels, epochs=epochs)""",

    """def evaluate_model(model, test_images, test_labels):
        test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
        print(f'\\nTest accuracy: {test_acc}')""",

    """def load_and_preprocess_data():
        (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
        train_images = train_images.astype('float32') / 255.0
        test_images = test_images.astype('float32') / 255.0
        return train_images, train_labels, test_images, test_labels"""
]

# Default resource allocation
default_resource = (1, 0, 0)  # CPU, GPU, TPU

def label_and_allocate(snippet, keywords):
    """Label the code snippet and allocate resources based on keywords."""
    resources = []
    for keyword, cpu, gpu, tpu in keywords:
        if keyword in snippet:
            resource = {
                'keyword': keyword,
                'CPU': bool(cpu),
                'GPU': bool(gpu),
                'TPU': bool(tpu)
            }
            resources.append(resource)
    return resources

# # Function to run each CNN snippet
# def run_snippet(snippet, device):
#     print(f"Executing snippet: {snippet[:30]}... on {device}")  # Print a preview of the snippet and the device
#     start_time = time.time()  # Start timing the execution
#     with tf.device(device):  # Allocate device for this snippet
#         exec(snippet)  # Execute the snippet
#     execution_time = time.time() - start_time  # Calculate execution time
#     return f"Execution completed in {execution_time:.2f} seconds on {device}."


def run_snippet(snippet, device):
    print(f"Executing snippet: {snippet[:30]}... on {device}")  # Print a preview of the snippet and the device
    start_time = time.time()  # Start timing the execution

    # Allocate device for this snippet
    with tf.device(device):
        if "load_and_preprocess_data" in snippet:
            train_images, train_labels, test_images, test_labels = load_and_preprocess_data()
        elif "create_cnn_model" in snippet:
            model = create_cnn_model((32, 32, 3))  # CIFAR-10 images are 32x32 pixels with 3 color channels
        elif "compile_and_train_model" in snippet:
            compile_and_train_model(model, train_images, train_labels, epochs=10)
        elif "evaluate_model" in snippet:
            evaluate_model(model, test_images, test_labels)

    execution_time = time.time() - start_time  # Calculate execution time
    return f"Execution completed in {execution_time:.2f} seconds on {device}."

def get_device(cpu, gpu, tpu):
    """Determine the device to use based on resource availability."""
    if tpu and tf.config.list_logical_devices('TPU'):
        return '/device:TPU:0'
    elif gpu and tf.config.list_physical_devices('GPU'):
        return '/device:GPU:0'
    else:
        return '/device:CPU:0'

# def master(file_path):
#     keywords = load_keywords_from_excel(file_path)
#     if keywords is None:
#         print("Failed to load keywords.")
#         return  # Stop further execution if keywords are not loaded

#     print("Loaded keywords:", keywords)  # Debug print to show loaded keywords
#     labeled_resources = []

#     # Label each snippet and allocate resources
#     for snippet in cnn_snippets:
#         resources = label_and_allocate(snippet, keywords)
#         labeled_resources.append((resources, snippet))

#     # Initialize TPU if any task requires it
#     tpu_required = any(resource[-1] for resource in keywords)  # Check if TPU is required (third item in the tuple)
#     if tpu_required:
#         tpu_available = init_tpu()  # Initialize TPU if needed
#         if not tpu_available:
#             print("TPU is required but unavailable. Proceeding with fallback to GPU or CPU.")

#     # Using ThreadPoolExecutor to simulate slave execution
#     with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
#         futures = {}
#         for resources, snippet in labeled_resources:
#             if resources:
#                 # Only extract CPU, GPU, and TPU values (ignore the keyword)
#                 resource_values = list(resources[0].values())[1:]  # Extract CPU, GPU, TPU values only
#                 device = get_device(*resource_values)
#             else:
#                 device = '/device:CPU:0'  # Default to CPU if no resource is matched

#             futures[executor.submit(run_snippet, snippet, device)] = resources

#         for future in concurrent.futures.as_completed(futures):
#             resources = futures[future]
#             try:
#                 result = future.result()
#                 print(f"Resources used: {resources} -> {result}")
#             except Exception as e:
#                 print(f"Error executing snippet: {e}")

def master(file_path):
    keywords = load_keywords_from_excel(file_path)
    if keywords is None:
        print("Failed to load keywords.")
        return  # Stop further execution if keywords are not loaded

    print("Loaded keywords:", keywords)  # Debug print to show loaded keywords
    labeled_resources = []

    # Label each snippet and allocate resources
    for snippet in cnn_snippets:
        resources = label_and_allocate(snippet, keywords)
        labeled_resources.append((resources, snippet))

    # Initialize TPU if any task requires it
    tpu_required = any(resource[-1] for resource in keywords)  # Check if TPU is required (third item in the tuple)
    if tpu_required:
        tpu_available = init_tpu()  # Initialize TPU if needed
        if not tpu_available:
            print("TPU is required but unavailable. Proceeding with fallback to GPU or CPU.")

    # Using ThreadPoolExecutor to simulate slave execution
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        futures = {}
        for resources, snippet in labeled_resources:
            if resources:
                # Only extract CPU, GPU, and TPU values (ignore the keyword)
                resource_values = list(resources[0].values())[1:]  # Extract CPU, GPU, TPU values only
                device = get_device(*resource_values)
            else:
                device = '/device:CPU:0'  # Default to CPU if no resource is matched

            futures[executor.submit(run_snippet, snippet, device)] = resources

        for future in concurrent.futures.as_completed(futures):
            resources = futures[future]
            try:
                result = future.result()
                print(f"Resources used: {resources} -> {result}")
            except Exception as e:
                print(f"Error executing snippet: {e}")


if __name__ == '__main__':
    # Specify the path to your Excel file
    excel_file_path = '/content/resource_allocation_dataset.xlsx'
    master(excel_file_path)


Columns in the Excel file: ['Keyword', 'CPU', 'GPU', 'TPU']
Loaded keywords: [('default', 1, 0, 0), ('import', 1, 0, 0), ('CNN', 0, 1, 1), ('RNN', 1, 0, 1), ('preprocess', 1, 1, 0), ('train', 0, 1, 1), ('eval', 1, 0, 0), ('transfer', 0, 1, 1), ('feature', 1, 1, 0), ('tune', 1, 0, 0), ('grid', 1, 0, 0), ('LSTM', 1, 0, 1), ('data_augmentation', 1, 1, 0), ('batch_normalization', 0, 1, 1), ('dropout', 1, 0, 0), ('optimization', 1, 1, 1), ('model_saving', 1, 0, 0), ('hyperparameter_search', 1, 1, 0), ('data_loading', 1, 1, 0), ('evaluation_metrics', 1, 0, 0), ('fine_tuning', 0, 1, 1), ('ensemble_learning', 1, 1, 0), ('tokenization', 1, 0, 0), ('backpropagation', 1, 0, 0), ('regularization', 1, 0, 0), ('transfer_learning', 0, 1, 1), ('seq2seq', 1, 0, 1), ('attention_mechanism', 0, 1, 1), ('model_export', 1, 0, 0), ('real_time_inference', 0, 1, 1), ('predictive_analysis', 1, 0, 0), ('class_weighting', 1, 1, 0), ('grid_search_cv', 1, 0, 0), ('cross_validation', 1, 1, 0), ('feature_scaling', 1,

In [8]:
#basic implementation works
import concurrent.futures
import pandas as pd
import tensorflow as tf
import time

# Initialize TPU if needed
def init_tpu():
    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print("TPU initialized.")
        return True
    except Exception as e:
        print(f"TPU initialization failed: {e}")
        return False

def load_keywords_from_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        df.columns = df.columns.str.strip()  # Strip whitespace from column names
        keywords = [(row['Keyword'], int(row['CPU']), int(row['GPU']), int(row['TPU'])) for _, row in df.iterrows()]
        print(f"Loaded keywords: {keywords}")
        return keywords
    except Exception as e:
        print(f"An error occurred: {e}")
        return None  # Return None in case of an error

def create_cnn_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

def compile_and_train_model(model, train_images, train_labels, epochs=10):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_images, train_labels, epochs=epochs)

def evaluate_model(model, test_images, test_labels):
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    print(f'\nTest accuracy: {test_acc}')

def load_and_preprocess_data():
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
    train_images = train_images.astype('float32') / 255.0
    test_images = test_images.astype('float32') / 255.0
    return train_images, train_labels, test_images, test_labels

def run_snippet(snippet, device):
    print(f"Executing snippet on {device}")  # Print device info
    start_time = time.time()

    with tf.device(device):
        if "load_and_preprocess_data" in snippet:
            return load_and_preprocess_data()
        elif "create_cnn_model" in snippet:
            return create_cnn_model((32, 32, 3))
        elif "compile_and_train_model" in snippet:
            model, train_images, train_labels = snippet['model'], snippet['train_images'], snippet['train_labels']
            compile_and_train_model(model, train_images, train_labels)
        elif "evaluate_model" in snippet:
            model, test_images, test_labels = snippet['model'], snippet['test_images'], snippet['test_labels']
            evaluate_model(model, test_images, test_labels)

    execution_time = time.time() - start_time
    return f"Execution completed in {execution_time:.2f} seconds on {device}."

def get_device(cpu, gpu, tpu):
    """Determine the device to use based on resource availability."""
    if tpu and tf.config.list_logical_devices('TPU'):
        return '/device:TPU:0'
    elif gpu and tf.config.list_physical_devices('GPU'):
        return '/device:GPU:0'
    else:
        return '/device:CPU:0'

def master(file_path):
    keywords = load_keywords_from_excel(file_path)
    if keywords is None:
        print("Failed to load keywords.")
        return

    labeled_resources = []

    for snippet in cnn_snippets:
        resources = label_and_allocate(snippet, keywords)
        labeled_resources.append((resources, snippet))

    tpu_required = any(resource[-1] for resource in keywords)
    if tpu_required:
        tpu_available = init_tpu()
        if not tpu_available:
            print("TPU is required but unavailable. Proceeding with fallback to GPU or CPU.")

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        futures = {}
        for resources, snippet in labeled_resources:
            if resources:
                resource_values = list(resources[0].values())[1:]
                device = get_device(*resource_values)
            else:
                device = '/device:CPU:0'

            futures[executor.submit(run_snippet, snippet, device)] = resources

        for future in concurrent.futures.as_completed(futures):
            resources = futures[future]
            try:
                result = future.result()
                print(f"Resources used: {resources} -> {result}")
            except Exception as e:
                print(f"Error executing snippet: {e}")

if __name__ == '__main__':
    excel_file_path = '/content/resource_allocation_dataset.xlsx'
    master(excel_file_path)


Loaded keywords: [('default', 1, 0, 0), ('import', 1, 0, 0), ('CNN', 0, 1, 1), ('RNN', 1, 0, 1), ('preprocess', 1, 1, 0), ('train', 0, 1, 1), ('eval', 1, 0, 0), ('transfer', 0, 1, 1), ('feature', 1, 1, 0), ('tune', 1, 0, 0), ('grid', 1, 0, 0), ('LSTM', 1, 0, 1), ('data_augmentation', 1, 1, 0), ('batch_normalization', 0, 1, 1), ('dropout', 1, 0, 0), ('optimization', 1, 1, 1), ('model_saving', 1, 0, 0), ('hyperparameter_search', 1, 1, 0), ('data_loading', 1, 1, 0), ('evaluation_metrics', 1, 0, 0), ('fine_tuning', 0, 1, 1), ('ensemble_learning', 1, 1, 0), ('tokenization', 1, 0, 0), ('backpropagation', 1, 0, 0), ('regularization', 1, 0, 0), ('transfer_learning', 0, 1, 1), ('seq2seq', 1, 0, 1), ('attention_mechanism', 0, 1, 1), ('model_export', 1, 0, 0), ('real_time_inference', 0, 1, 1), ('predictive_analysis', 1, 0, 0), ('class_weighting', 1, 1, 0), ('grid_search_cv', 1, 0, 0), ('cross_validation', 1, 1, 0), ('feature_scaling', 1, 1, 0), ('loss_function', 1, 0, 0), ('convolutional_layer', 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Resources used: [{'keyword': 'preprocess', 'CPU': True, 'GPU': True, 'TPU': False}, {'keyword': 'train', 'CPU': False, 'GPU': True, 'TPU': True}] -> (array([[[[0.23137255, 0.24313726, 0.24705882],
         [0.16862746, 0.18039216, 0.1764706 ],
         [0.19607843, 0.1882353 , 0.16862746],
         ...,
         [0.61960787, 0.5176471 , 0.42352942],
         [0.59607846, 0.49019608, 0.4       ],
         [0.5803922 , 0.4862745 , 0.40392157]],

        [[0.0627451 , 0.07843138, 0.07843138],
         [0.        , 0.        , 0.        ],
         [0.07058824, 0.03137255, 0.        ],
         ...,
         [0.48235294, 0.34509805, 0.21568628],
         [0.46666667, 0.3254902 , 0.19607843],
         [0.47843137, 0.34117648, 0.22352941]],

        [[0.09803922, 0.09411765, 0.08235294],
         [0.0627451 , 0.02745098, 0.        ],
         [0.19215687, 0.10588235, 0.03137255],
         ...,
         [0.4627451 , 0.32941177, 0.19607843],